# The RUV Package

## Looking to clean your data?  Learn how to Remove Unwanted Variation with R

## useR! 2018 Brisbane

This notebook accompanies Session 1 Slides

# The Data

In [ ]:
load("gender.rda")
ls()
# Y.raw:  Summarized by RMA, but otherwise not preprocessed
# Y.norm: Background corrected and quantile normalized

In [ ]:
Y = Y.norm
Y[1:5, 1:5]

In [ ]:
head(sampleinfo)

In [ ]:
head(geneinfo)

In [ ]:
# Load the ruv package
library(ruv)
# Graphics 
library(ggplot2)
library(gridExtra)
gg_additions = list(aes(color=sampleinfo$region, 
                        shape=sampleinfo$lab, 
                        size=5, alpha=.7), 
                    labs(color="Brain Region", 
                         shape="Laboratory"),
                    scale_size_identity(guide="none"),
                    scale_alpha(guide="none"),
                    theme(legend.text=element_text(size=12),
                          legend.title=element_text(size=16)),
                    guides(color = guide_legend(override.aes = list(size = 4)),
                           shape = guide_legend(override.aes = list(size = 4))),
                    scale_color_manual(values=c("darkorchid3", "darkorange2", "dodgerblue3"))
                   ) 
options(repr.plot.width=8, repr.plot.height=6)

In [ ]:
ruv_svdplot(Y) + gg_additions # Technical note: centers columns by default

In [ ]:
ruv_svdplot(residop(scale(Y,scale=FALSE), svd(scale(Y,scale=FALSE))$u[,1:5])) + gg_additions 

In [ ]:
ruv_svdplot(RUVIII(Y, replicate.matrix(sampleinfo[,c("patient", "region")]), geneinfo$spikectl, k=10)) + gg_additions

# Example Analysis (Regression)

In [ ]:
fit = RUVrinv(Y, sampleinfo$gender, geneinfo$spikectl)
fit.summary = ruv_summary(Y, fit, sampleinfo, geneinfo)
head(fit.summary$C)

In [ ]:
ruv_hist(fit.summary)

In [ ]:
ruv_ecdf(fit.summary)

In [ ]:
ruv_ecdf(fit.summary, power=1/4)

In [ ]:
genecoloring = list(
aes(color=genetype),
scale_color_manual(name="Gene Category", 
                   values=alpha(c("green", "gray", "yellow", "palevioletred1", "purple", "deepskyblue"), 
                                c(     .2,    .15,        1,                1,        1,             1)))
)

In [ ]:
ruv_ecdf(fit.summary) + genecoloring

In [ ]:
ruv_rankplot(fit.summary, "pctl")  # "pctl" is a column in "geneinfo".  Genes from X/Y chrom.

In [ ]:
ruv_rankplot(fit.summary, "pctl") + coord_cartesian(xlim=c(0,50), ylim=c(0,25))

In [ ]:
ruv_projectionplot(fit.summary) + genecoloring

In [ ]:
ruv_volcano(fit.summary) + genecoloring

In [ ]:
ruv_varianceplot(fit.summary) + genecoloring

In [ ]:
fit.summary.evar = ruv_summary(Y, fit, sampleinfo, geneinfo, p.type="evar")
ruv_varianceplot(fit.summary.evar) + genecoloring

# Did we help?

In [ ]:
# RUV4 with k = 0 for no adjustment
# Equivalent to a Limma Analysis
fit.unadj = RUV4(Y, sampleinfo$gender, geneinfo$spikectl, 0)       
fit.summary.unadj = ruv_summary(Y, fit.unadj, sampleinfo, geneinfo)  
# Make a list of plots to compare side-by-side
plots = list(
  ruv_hist(fit.summary.unadj),
  ruv_hist(fit.summary),
  ruv_rankplot(fit.summary.unadj, "pctl") + 
    coord_cartesian(xlim=c(0,50), ylim=c(0,25)),
  ruv_rankplot(fit.summary, "pctl") + 
    coord_cartesian(xlim=c(0,50), ylim=c(0,25))
)

In [ ]:
grid.arrange(grobs=plots)

# Example Analyses (Global Adjustments)

# Example 1

## Spike-in Negative Controls and Technical Replicates

In [ ]:
ruv_svdplot(Y) + gg_additions

In [ ]:
M = replicate.matrix(sampleinfo[,c("patient", "region")])
YIII.spike.tech = RUVIII(Y, M, geneinfo$spikectl, k=10)

In [ ]:
ruv_svdplot(YIII.spike.tech) + gg_additions

In [ ]:
# This time, set average=TRUE
YIII.spike.tech.avg = RUVIII(Y, M, geneinfo$spikectl, k=10, average=TRUE)
# Create "metadata" for the rows of YIII.spike.tech.avg
sampleinfo.spike.tech.avg = collapse.replicates(sampleinfo, M)
head(sampleinfo.spike.tech.avg)

In [ ]:
ruv_svdplot(YIII.spike.tech.avg) + 
  aes(color=sampleinfo.spike.tech.avg$region)

# Example 2

## Plotting just the X/Y genes

In [ ]:
ruv_svdplot(Y[,geneinfo$pctl]) + gg_additions

In [ ]:
gg_gender_region = list(aes(color=sampleinfo$region, 
                            shape=sampleinfo$gender, 
                            size=3, alpha=1, stroke=2), 
                        labs(color="Brain Region", 
                             shape="Gender"),
                        scale_size_identity(guide="none"),
                        scale_alpha(guide="none"),
                        scale_shape_manual(values = c("male" = 5, "female" = 3)),
                        theme(legend.text=element_text(size=12),
                              legend.title=element_text(size=16)),
                        guides(color = guide_legend(override.aes = list(size = 4)),
                               shape = guide_legend(override.aes = list(size = 4))),
                        scale_color_manual(values=c("darkorchid3", "darkorange2", "dodgerblue3"))
                       ) 

In [ ]:
ruv_svdplot(Y[,geneinfo$pctl]) + gg_gender_region

In [ ]:
ruv_svdplot(YIII.spike.tech[,geneinfo$pctl]) + gg_gender_region

# Example 3

## Just the X/Y genes, Continued

In [ ]:
M = replicate.matrix(sampleinfo[,c("patient")])
YIII.hk.bio = RUVIII(Y, M, geneinfo$hkctl, k=10)

In [ ]:
ruv_svdplot(YIII.hk.bio[,geneinfo$pctl]) + gg_gender_region

In [ ]:
# Create a design matrix for brain region:
region_mat = design.matrix(sampleinfo$region)
# Regress it out from the "technical-adjusted" dataset
YIII.spike.tech.region_regression = residop(YIII.spike.tech, region_mat)

In [ ]:
ruv_svdplot(YIII.spike.tech.region_regression[,geneinfo$pctl]) + gg_gender_region

In [ ]:
gg_gender_region_nooutlier = list(aes(color=sampleinfo$region[-15], 
                            shape=sampleinfo$gender[-15], 
                            size=3, alpha=1, stroke=2), 
                        labs(color="Brain Region", 
                             shape="Gender"),
                        scale_size_identity(guide="none"),
                        scale_alpha(guide="none"),
                        scale_shape_manual(values = c("male" = 5, "female" = 3)),
                        theme(legend.text=element_text(size=12),
                              legend.title=element_text(size=16)),
                        guides(color = guide_legend(override.aes = list(size = 4)),
                               shape = guide_legend(override.aes = list(size = 4))),
                        scale_color_manual(values=c("darkorchid3", "darkorange2", "dodgerblue3"))
                       ) 

In [ ]:
ruv_svdplot(YIII.spike.tech.region_regression[-15,geneinfo$pctl]) + gg_gender_region_nooutlier

In [ ]:
ruv_svdplot(YIII.spike.tech.region_regression[-15,geneinfo$pctl], k=3:4) + gg_gender_region_nooutlier

# Final Example

In [ ]:
M = replicate.matrix(sampleinfo[,c("region")])
newY3 = RUVIII(Y, M, geneinfo$hkctl, k=10)

In [ ]:
ruv_svdplot(newY3) + gg_additions

In [ ]:
M = replicate.matrix(sampleinfo[,c("region")], burst=c("cerebellum", "D.L.P.F..cortex"))
newY3 = RUVIII(Y, M, geneinfo$hkctl, k=10)

In [ ]:
ruv_svdplot(newY3) + gg_additions

# Examples with Shiny

# Balanced Design

In [ ]:
library(ruv)
library(shiny)
library(colourpicker)
load("gender.rda")
Y = Y.norm
ruv_shiny(Y,sampleinfo,geneinfo,options=list(port=3840,host="0.0.0.0"))

# Imbalanced Design

In [ ]:
keep = rep(T,nrow(Y))
keep[sampleinfo$lab=="Davis" & sampleinfo$gender=="male"] = FALSE
keep[sampleinfo$lab=="Michigan" & sampleinfo$gender=="female"] = FALSE
Y.imb = Y[keep,]
sampleinfo.imb = sampleinfo[keep,]
ruv_shiny(Y.imb,sampleinfo.imb,geneinfo,options=list(port=3840,host="0.0.0.0"))

In [ ]:
keep = rep(T,nrow(Y))
keep[sampleinfo$lab=="Davis" & sampleinfo$gender=="male"] = FALSE
keep[sampleinfo$lab=="Michigan" & sampleinfo$gender=="female"] = FALSE
Y.imb = Y.raw[keep,]
sampleinfo.imb = sampleinfo[keep,]
ruv_shiny(Y.imb,sampleinfo.imb,geneinfo,options=list(port=3840,host="0.0.0.0"))

# Brain Region

In [ ]:
ruv_shiny(Y.raw,sampleinfo,geneinfo,options=list(port=3840,host="0.0.0.0"))

In [ ]:
newY = RUVI(Y.raw, 1, geneinfo$spikectl)
M = replicate.matrix(sampleinfo[,c("patient", "region")])
newY = RUVIII(newY, M, geneinfo$spikectl, k=4, average=TRUE)
newsampleinfo = collapse.replicates(sampleinfo, M)
fit = RUV4(newY, newsampleinfo$cortex, rep(TRUE,ncol(newY)), k=1)
fit = ruv_summary(newY, fit, newsampleinfo, geneinfo)

In [ ]:
ruv_ecdf(fit, uniform.lines=seq(0,1,by=.1))

In [ ]:
mean(fit$C$F.p > .25)
mean(fit$C$F.p.BH > .5)

In [ ]:
ectl = colnames(newY) %in% rownames(fit$C)[fit$C$F.p.BH > .5]
geneinfo = cbind(geneinfo, neg.cer=ectl)

In [ ]:
ruv_shiny(Y.raw, sampleinfo, geneinfo)